In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import TensorBoard

from PIL import Image

In [ ]:
# Load TensorBoard to monitor training activity on realtime.

%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
# Load dataset images into the DataFrame.

dataset = []

for emotion in os.listdir('dataset'):
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}')
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
# Normalizacion of the data.

X = np.array([MinMaxScaler().fit_transform(e) for e in df['pixels']])
Y = LabelBinarizer().fit_transform(df['emotion'])

In [ ]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT = X.shape
_, NUM_LABELS = Y.shape

NUM_CHANNELS = 1

In [ ]:
# Create a Convolutional model.

modelConv = Sequential([
    Conv2D(32, (3,3), padding="Same", activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS)),
    BatchNormalization(),
    Conv2D(32, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Conv2D(64, (3,3), padding="Same", activation='relu'),
    BatchNormalization(),
    Conv2D(64, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Conv2D(128, (3,3), padding="Same", activation='relu'),
    BatchNormalization(),
    Conv2D(128, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelConv.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
# Train the convolutional model.

modelConv.fit(X_train, Y_train, batch_size=32, epochs=50, validation_data=(X_test, Y_test), callbacks=[TensorBoard(log_dir="logs/modelConv")])

In [ ]:
score, acc = modelConv.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [ ]:
# Load dataset images into the DataFrame and convert into RGB (required by MobileNetV2)

dataset = []

for emotion in os.listdir('dataset'):
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}').resize((128, 128), resample=Image.Resampling.LANCZOS).convert("RGB")
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
# Normalizacion of the data.

X = np.array((df['pixels']/255).values.tolist())
Y = LabelBinarizer().fit_transform(df['emotion'])

In [ ]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X.shape

In [ ]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS = X.shape
_, NUM_LABELS = Y.shape

In [ ]:
# Create a MobileNetV2 based model.

base_model = MobileNetV2(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                         include_top=False,
                         weights='imagenet')

#base_model.trainable = False

modelMobNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelMobNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
# Train the MobileNetV2 based model.

modelMobNet.fit(X_train, Y_train, batch_size=32, epochs=50,  validation_data=(X_test, Y_test), callbacks=[TensorBoard(log_dir="logs/modelMobNet")])

In [ ]:
score, acc = modelMobNet.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [ ]:
# Create a ResNet50 based model.

base_model = model = ResNet50(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                                               include_top=False,
                                               weights='imagenet')

#base_model.trainable = False

modelResNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelResNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
# Train the ResNet50 based model.

modelResNet.fit(X_train, Y_train, batch_size=32, epochs=50,  validation_data=(X_test, Y_test), callbacks=[TensorBoard(log_dir="logs/modelResNet")])